# R2B2 Basics

The `r2b2` library provides a simple framework for working with risk-limiting audits. 

## Modules

Data is divided into 3 main object classes:

- `Election` - Maintain data specific to an election and the collection of contests within it.
- `Contest` - Maintain data specific to a single contest such as candidates, tally, etc.
- `Audit` - An abstract class which gives a structure for a risk-limiting audit and hold the parameters of an audit such as risk limit, sampling method, etc.

In addition to these modules, `r2b2` also provides a command line tool.

## Specific Audits

The `Audit` class is an abstract class that cannot be instantiated. Instead, specific audits (such as BRAVO, BRLA, etc.) are implemented as subclasses of `Audit` which can then be instantiated.


# A Simple Example

Let's learn the basic usage of `r2b2` with a simple example! 

Suppose you have data from a certain contest and you would like to compare the possible results from different audits on the contest. For this example, we will use the Bayesian Risk-Limiting audit.

First, we will need to import the necessary packages.

In [1]:
from r2b2.contest import Contest, ContestType
from r2b2.brla import BayesianRLA as BRLA

## Creating a Contest

First we create a plurality contest with the following data:

||||
|:---|:---|:---|
| | Ballots Cast | 1000|
|Tally | A | 700 |
||B | 300 |
|| Reported Winner | A |


In [2]:
example_contest = Contest(1000, {'A': 700, 'B': 300}, 1, ['A'], ContestType.PLURALITY)

# TODO: add print(contest) once cli branch pulled in

## Creating an Audit

Let's start by creating just one audit to examine what we can do with the audit class.

Our audit will have the following parameters:

|||
|:---|:---|
|Risk Limit| 10% |
|Maximum Fraction of Ballots to draw | 30% |


In [3]:
audit1 = BRLA(0.1, 0.3, example_contest)

### Using the Audit

Assume you know what the audit execution will be before the audit beings: You will to draw samples from the contest ballots in quantities of 100 for 3 rounds, giving total sample sizes of 100, 200, and 300. 

Then, before the audit begins, we can compute the minimum number of ballots for the reported winner which need to be found in each rounds sample in order to stop the audit at that round. We will use the `compute_min_winner_ballots(rouunds)` to compute these minimums.

In [4]:
rounds = [100, 200, 300]
min_winner_ballots1 = audit1.compute_min_winner_ballots(rounds)
print(rounds)
print(min_winner_ballots1)

[100, 200, 300]
[63, 118, 172]


If you do not know the audit execution ahead of time, then you can compute the risk of a given sample and compare to the esired risk limit during execution. 

Let's say we draw a first round of 94 ballots and find 39 ballots for candidate A. Then we use `compute_risk(votes_for_winner, round)` to get the risk level of this round.

In [4]:
current_risk = audit1.compute_risk(39, 94)
print('Risk: ', current_risk)
print('Stopping condition met? ', current_risk < 0.1)

Risk:  0.9571428048908984
Stopping condition met?  False


As you can see, we do not yet meet hte stopping condition. For the next round, you draw a sample of 154 ballots (for a total of 248) and find 113 ballots for candidate A (for a total of 152) and check the risk lmit again.

In [5]:
current_risk = audit1.compute_risk(152, 248)
print('Risk: ', current_risk)
print('Stopping condition met? ', current_risk < 0.1)

Risk:  0.0015727415171509895
Stopping condition met?  True


**You meet the stopping condition!**

## Writing an Audit Script

Now that you know how to use Contests and Audits, let's write a simple script to create a table comparing the minimum number of winner ballots needed in each round for different risk limits. 

**Example Use Case:**
> You are about to audit a contest and have decided you are willing to sample at most 300 ballots across 3 rounds (with cumulative sizes 100, 200, and 300) and must determine what risk limit to use.

In [7]:
# Table data
risk_limits = [0.01, 0.02, 0.05, 0.1, 0.15, 0.2]
rounds = [100, 200, 300]
results = {}

# Create an audit for each of the risk limits
# Find the minimum ballots needed to stop for each round size
# Store minimums in results
for r in risk_limits:
    audit = BRLA(r, 0.3, example_contest)
    results[r] = audit.compute_min_winner_ballots(rounds)

In [27]:
# Print table with minimal formatting
print("Risk / Rounds")
print("     | ", ' | '.join('{:<5}'.format(r) for r in rounds))
print('---------------------------')
for k,v in results.items():
    left = '{:.2f} | '.format(k)
    right = ' | '.join('{:<5}'.format(i) for i in v)
    print(left, right)

Risk / Rounds
     |  100   | 200   | 300  
---------------------------
0.01 |  67    | 123   | 177  
0.02 |  66    | 122   | 176  
0.05 |  65    | 120   | 174  
0.10 |  63    | 118   | 172  
0.15 |  62    | 117   | 171  
0.20 |  62    | 117   | 170  


You can now see the different minimum stopping values you would need for different risk limits for an audit with the given round schedule. Intuitively, the number of winner ballots needed to stop in each round decreases as the risk limit increases. 


Now let us see how we might compare round sizes given a risk limit.

**Example Use Case:**
> You are about to audit a contest and have decided to use a 5% risk limit and are willing to sample at most 300 ballots across 5 rounds. What sample sizes should you use?

In [43]:
# Create an audit object for the given risk limit and max sample size
audit = BRLA(0.05, 0.3, example_contest)

# Generate a list of stopping values from the min to max sample size
results2 = audit.compute_all_min_winner_ballots()
rounds = [i for i in range(audit.min_sample_size, 301)]

In [44]:
# Print ful list of round sizes
for i in range(len(rounds)):
    print('{:<4} | {:<4}'.format(rounds[i], results2[i]))

7    | 7   
8    | 8   
9    | 9   
10   | 9   
11   | 10  
12   | 11  
13   | 12  
14   | 12  
15   | 13  
16   | 14  
17   | 14  
18   | 15  
19   | 16  
20   | 16  
21   | 17  
22   | 18  
23   | 18  
24   | 19  
25   | 20  
26   | 20  
27   | 21  
28   | 22  
29   | 22  
30   | 23  
31   | 24  
32   | 24  
33   | 25  
34   | 25  
35   | 26  
36   | 27  
37   | 27  
38   | 28  
39   | 29  
40   | 29  
41   | 30  
42   | 30  
43   | 31  
44   | 32  
45   | 32  
46   | 33  
47   | 33  
48   | 34  
49   | 35  
50   | 35  
51   | 36  
52   | 36  
53   | 37  
54   | 38  
55   | 38  
56   | 39  
57   | 39  
58   | 40  
59   | 41  
60   | 41  
61   | 42  
62   | 42  
63   | 43  
64   | 44  
65   | 44  
66   | 45  
67   | 45  
68   | 46  
69   | 47  
70   | 47  
71   | 48  
72   | 48  
73   | 49  
74   | 50  
75   | 50  
76   | 51  
77   | 51  
78   | 52  
79   | 52  
80   | 53  
81   | 54  
82   | 54  
83   | 55  
84   | 55  
85   | 56  
86   | 56  
87   | 57  
88   | 58  
89   | 58  
90  

Now we have a list of all the possible round sizes and their minimum winner ballots to stop. This list is very long, to help decide which round sizes to use, let's assume a few more constraints:

- You won't sample any fewer than 20 ballots
- You only want sample sizes that are multiples of 20

Now we simply retrieve the relevant round sizes from the list of results and print them in a table. Let's also provide some additional information: the minimum's proportion of the sample size.

In [62]:
better_results = {}
# Get multiples of 20
for i in range(rounds.index(20), len(rounds), 20):
    better_results[rounds[i]] = results2[i]

# Print better results table
print('Round | Mininum Stopping | Proportion')
print('------|------------------|-----------')
for k,v in better_results.items():
    print('{:<5} | {:<16} | {:.3f}'.format(k ,v, v/k))

Round | Mininum Stopping | Proportion
------|------------------|-----------
20    | 16               | 0.800
40    | 29               | 0.725
60    | 41               | 0.683
80    | 53               | 0.662
100   | 65               | 0.650
120   | 76               | 0.633
140   | 87               | 0.621
160   | 98               | 0.613
180   | 109              | 0.606
200   | 120              | 0.600
220   | 131              | 0.595
240   | 142              | 0.592
260   | 152              | 0.585
280   | 163              | 0.582
300   | 174              | 0.580


Now you have a digestible list of round sizes with some additional information to help decide on the round sizes.